In [105]:
import pandas as pd
from cplex import Cplex

In [106]:
finished_raw_data = pd.read_excel('finished_raw_ratio_rate.xlsx', engine = 'openpyxl')
sales_data = pd.read_excel('Average_sales_product.xlsx', engine = 'openpyxl')
raw_rate = pd.read_excel('raw_rate.xlsx', engine = 'openpyxl')

In [107]:
print(finished_raw_data.columns)
print(sales_data.columns)
print(raw_rate.columns)

Index(['Unnamed: 0', 'goods', 'raw', 'xqty/unit', 'xrate'], dtype='object')
Index(['Unnamed: 0', 'xitem', 'xdesc', 'xqty', 'xval', 'xlineamt'], dtype='object')
Index(['Unnamed: 0', 'raw', 'xrate'], dtype='object')


In [108]:
budget = sales_data['xval'].sum() * 1.0
print(budget)

992123.87


In [109]:
# load all the products sold and average sales qty
products = sales_data['xitem'].unique()
average_sales = dict(zip(sales_data['xitem'], sales_data['xqty']))

# load raw materials relations 
raw_materials = raw_rate['raw'].unique()
raw_material_cost = dict(zip(raw_rate['raw'], raw_rate['xrate']))

In [110]:
#problem setup
problem = Cplex()
problem.set_problem_type(Cplex.problem_type.LP)
# problem.objective.set_sense(problem.objective.sense.minimize)
problem.objective.set_sense(problem.objective.sense.maximize)


In [111]:
# add decision variables for the problem
raw_material_vars = {material: problem.variables.add(
names=[f"x_{material}"], lb=[0], ub=[float('inf')])[0] for material in raw_materials}
# print(raw_material_vars)

In [112]:
# complete coefficients and indices and setup objective 
cost_coefficients = [raw_material_cost[material] for material in raw_materials]
variable_indices = [raw_material_vars[material] for material in raw_materials]

# minimize cost
# problem.objective.set_linear(zip(variable_indices, cost_coefficients))
problem.objective.set_linear([(raw_material_vars[material], 1) for material in raw_materials])

In [113]:
# constraints 
for product in products:
    required_materials = finished_raw_data[finished_raw_data['goods'] == product]
    product_demand = average_sales[product]
    
    for material in raw_materials:
        filtered_material = required_materials[required_materials['raw'] == material]
        
        if not filtered_material.empty:
            required_units = filtered_material['xqty/unit'].values[0] * product_demand
            if required_units > 0:
                    problem.linear_constraints.add(
                    lin_expr=[[ [raw_material_vars[material]], [required_units] ]], 
                    senses=['L'],
                    rhs=[required_units],
                    names=[f"Demand_Constraint_{product}_{material}"]
                )
        else:
            continue

In [114]:
total_cost_expr = [[list(raw_material_vars.values()), [raw_material_cost[material] for material in raw_materials]]]
problem.linear_constraints.add(
    lin_expr=total_cost_expr,
    senses=['L'],
    rhs=[budget],
    names=["Budget_Constraint"]
)

range(737, 738)

In [115]:
problem.write("inventory_problem.lp")
problem.solve()
solution = {}
for material in raw_materials:
    index = raw_material_vars[material]
    solution[material] = problem.solution.get_values(index)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 738 rows and 121 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.21 ticks)


In [116]:
print("Optimal Quantities of Raw Materials to Purchase:")
for material, quantity in solution.items():
    print(f"{material}: {quantity}")
print(f"Total Cost: {problem.solution.get_objective_value()}")

Optimal Quantities of Raw Materials to Purchase:
RZ000001: 1.0
RZ000002: 1.0
RZ000003: 1.0
RZ000004: 1.0
RZ000005: 1.0
RZ000006: 1.0
RZ000007: 1.0
RZ000009: 1.0
RZ000010: 1.0
RZ000012: 1.0
RZ000021: 1.0
RZ000030: 1.0
RZ000138: 1.0
RZ000148: 0.9999999999999999
RZ000157: 1.0
RZ000158: 0.9999999999999999
RZ000159: 1.0
RZ000160: 1.0
RZ000172: 1.0
RZ000173: 1.0
RZ000176: 1.0
RZ000177: 1.0
RZ000178: 1.0
RZ000180: 1.0
RZ000181: 1.0
RZ000193: 1.0
RZ000199: 1.0
RZ000223: 1.0
RZ000228: 1.0
RZ000230: 1.0
RZ000235: 1.0
RZ000241: 1.0
RZ000254: 1.0
RZ000270: 1.0
RZ000273: 1.0
RZ000274: 1.0
RZ000275: 1.0
RZ000276: 1.0
RZ000277: 1.0
RZ000278: 1.0
RZ000279: 1.0
RZ000282: 1.0
RZ000303: 1.0
RZ000304: 1.0
RZ000305: 0.9999999999999999
RZ000309: 1.0
RZ000310: 1.0
RZ000311: 1.0
RZ000312: 1.0
RZ000315: 1.0
RZ000316: 1.0
RZ000317: 0.9999999999999999
RZ000319: 0.9999999999999999
RZ000320: 0.9999999999999999
RZ000325: 1.0
RZ000326: 1.0
RZ000332: 1.0
RZ000343: 1.0
RZ000344: 1.0
RZ000345: 1.0
RZ000346: 1.0
RZ00034